In [1]:
import math
from collections import deque
from collections import namedtuple

In [2]:
AssignedJob = namedtuple("AssignedJob", ["worker", "started_at"])

In [3]:
def heap_insert(arr, num, num_threads):
    if arr == []:
        arr.append(0)
        arr.append(num)
        return arr
    
    num_i = len(arr)
    while num_i != 0:
        parent_i = math.floor((num_i-1)/num_threads)
        num += arr[parent_i]
        num_i = parent_i
        
    arr.append(num)
    
    return arr

In [4]:
def sift_down(arr, num_i, num_threads):
    max_i = len(arr) - 1
    
    while True:
        if num_i == max_i:
            return arr
        
        first_child_i = num_i * num_threads + 1
        last_child_i = num_i * num_threads + num_threads
        num_child = num_threads
        
        if first_child_i > max_i:
            return arr
        if last_child_i > max_i:
            num_child -= (last_child_i - max_i)

        if num_child == 1:
            child_i = num_threads*num_i+1
            if arr[num_i] > arr[child_i]:
                arr[num_i], arr[child_i] = arr[child_i], arr[num_i]
                num_i = child_i
            else:
                return arr
        else:      
            the_min = min(arr[num_i], *[arr[num_threads*num_i+i] for i in range(1, num_child+1)])
            if the_min == arr[num_i]:
                return arr
            for i in range(1, num_child+1):
                if arr[num_threads*num_i+i] == the_min:
                    arr[num_threads*num_i+i], arr[num_i] = arr[num_i], arr[num_threads*num_i+i]
                    num_i = num_threads*num_i+i
                    break
    

In [5]:
def extract_min(arr, num_threads):
    
    if arr == []:
        return
    
    the_min = arr[0]
    arr[0], arr[-1] = arr[-1], arr[0]
    arr.pop()
    arr = sift_down(arr, 0, num_threads)
    
    return the_min, arr

In [6]:
def next_free_time(jobs, num_threads):
    arr = []
    for number in jobs:
        arr = heap_insert(arr, number, num_threads)
    
    free_time_lst = []
    while arr != []:
        the_min, arr = extract_min(arr, num_threads)
        free_time_lst.append(the_min)
        
    return free_time_lst

In [7]:
def assign_jobs(workers, jobs):
    free_time_lst = next_free_time(jobs, workers)
    worker_lst = deque()
    ans = []
    
    start_time, free_time_lst = extract_min(free_time_lst, workers)
    for i in range(workers):
        ans.append(AssignedJob(i, start_time))
        worker_lst.append(i)
      
    num_job = len(jobs) - workers
    while num_job > 0:
        start_time, free_time_lst = extract_min(free_time_lst, workers)
        worker = worker_lst.popleft()
        worker_lst.append(worker)
        ans.append(AssignedJob(worker, start_time))
        num_job -= 1
        
    return ans

In [8]:
jobs = [1,2,3,4,5]
workers = 2
assign_jobs(workers, jobs)

[AssignedJob(worker=0, started_at=0),
 AssignedJob(worker=1, started_at=0),
 AssignedJob(worker=0, started_at=1),
 AssignedJob(worker=1, started_at=2),
 AssignedJob(worker=0, started_at=4)]

In [9]:
jobs = [1]*20
workers = 4
next_free_time(jobs, workers)

[0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [10]:
def main():
    n_workers, n_jobs = map(int, input().split())
    jobs = list(map(int, input().split()))
    assert len(jobs) == n_jobs

    assigned_jobs = assign_jobs(n_workers, jobs)

    for job in assigned_jobs:
        print(job.worker, job.started_at)